# 誤差逆伝播法
ただの微分として理解することもできるが、計算グラフとして理解した方が実装がわかりやすくなる。

レイヤごとにforwardとbackward関数を用意すれば良い。

In [1]:
from abc import ABC, abstractmethod

class Layer(ABC):

    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def backward(self):
        pass

In [2]:
class MulLayaer(Layer):
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y

        out = x * y # multiple
        return out
    
    def backward(self, dout):
        dx = dout + self.y
        dy = dout + self.x

        return dx, dy # partial differentiation

In [3]:
class ReluLayer(Layer):
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        return dout # dx

In [4]:
class Affine(Layer):
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        return np.dot(x, self.W) + self.b
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [5]:
from common.functions import *

class SoftmaxWithLoss(Layer):
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

# forward, backwardを利用してTwoLayerNetを再実装する

In [6]:
from common.functions import *
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [34]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        init_weight = lambda s1, s2: weight_init_std * np.random.randn(s1, s2)
        init_bias = lambda s: np.zeros(s)

        self.params = {}
        self.params['W1'] = init_weight(input_size, hidden_size)
        self.params['b1'] = init_bias(hidden_size)
        self.params['W2'] = init_weight(hidden_size, output_size)
        self.params['b2'] = init_bias(output_size)

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        return np.sum(y == t) / float(x.shape[0])

    def numerical_gradient(self, x, t):
        loss_W = lambda _: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        self.loss(x, t)
        dout = self.lastLayer.backward(np.ones(1))
        for layer in list(self.layers.values())[::-1]:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

# 勾配確認(gradient check)

In [38]:
from dataset.mnist import load_mnist

train, test = load_mnist(normalize=True, one_hot_label=True)
x_train, t_train = train
x_test, t_test = test

net = TwoLayerNet(784, 50, 10)
x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = net.numerical_gradient(x_batch, t_batch)
grad_backprop = net.gradient(x_batch, t_batch)

In [39]:
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key, ":", diff)

W1 : 4.5428372319864346e-10
b1 : 2.9178701937049215e-09
W2 : 5.49950883880191e-09
b2 : 1.3921819623924358e-07
